# 旋转矩阵笔记

在三维上的旋转矩阵实际上都是二维平面的推广。在二维上有：

$$\begin{bmatrix}
x'\\y'
\end{bmatrix} = 
\begin{bmatrix}
cos\theta & -sin\theta\\ 
sin\theta & cos\theta
\end{bmatrix}
\begin{bmatrix}
x\\y 
\end{bmatrix}$$

表示一个点(x,y)绕原点旋转\theta 度，旋转前后点与原点距离不变。

在三维中绕原点旋转推广成为绕轴旋转，旋转当中点与绕的轴距离不变，旋转形成的平面与轴正交。
按照类似的方法即可得到绕xyz轴旋转的旋转矩阵，在此先把三个旋转矩阵记为 $R_xR_yR_z$ 。

首先编写生成旋转矩阵生成公式。

In [1]:
import numpy as np

def rotateX(theta):
    c = np.cos(np.radians(theta))
    s = np.sin(np.radians(theta))
    return np.mat([
        [1, 0, 0],
        [0, c, -s],
        [0, s, c]
    ])

def rotateY(theta):
    c = np.cos(np.radians(theta))
    s = np.sin(np.radians(theta))
    return np.mat([
        [c, 0, s],
        [0, 1, 0],
        [-s, 0, c]
    ])

def rotateZ(theta):
    c = np.cos(np.radians(theta))
    s = np.sin(np.radians(theta))
    return np.mat([
        [c, -s, 0],
        [s, c, 0],
        [0, 0, 1]
    ])

## 三维旋转矩阵的性质

### 旋转矩阵的复合不符合交换律

举个例子好了：

In [2]:
a = np.mat([1, 2, 3]).T
a1 = rotateY(20) * rotateZ(30) * rotateX(10) * a
a2 = rotateX(10) * rotateY(20) * rotateZ(30) * a
print(a1)
print(a2)
print((a1 == a2).all())

[[1.26239959]
 [1.75458586]
 [3.05414075]]
[[0.90016549]
 [1.7006563 ]
 [3.20896716]]
False


一个更直观的角度解释这个问题

![](img/001_001.png)

**旋转和坐标不一样，每次旋转的结果依赖上一次旋转的结果，不符合交换律。**[ref](https://math.stackexchange.com/questions/2016937/why-are-rotational-matrices-not-commutative)

### 旋转矩阵是正交矩阵

In [3]:
m = rotateY(20) * rotateZ(30) * rotateX(10)
print(m.I)
print(m.T)
((m.I - m.T) < 1e-10).all()

[[ 0.81379768  0.5        -0.29619813]
 [-0.40331711  0.85286853  0.33158796]
 [ 0.41841204 -0.15038373  0.89572099]]
[[ 0.81379768  0.5        -0.29619813]
 [-0.40331711  0.85286853  0.33158796]
 [ 0.41841204 -0.15038373  0.89572099]]


True

### 旋转矩阵与欧拉角

欧拉角，便是可以用3个角度去描述任意一个物体的旋转。例如，在右手坐标系定义顺序 $zxy$，则欧拉角$(a,b,c)$，
先顺着物体自己的坐标系（刚体坐标轴）z轴旋转a度，此时，物体坐标系的xyz轴分别旋转a度，显然z轴在这次旋转不动。接着顺着x轴y轴，
做对应旋转。

值得留意的是，刚开始定义的旋转矩阵是根据世界坐标系来定义的，这和上面欧拉角定义有所不同。显然上面的旋转也能用世界坐标系来表示，用旋转矩阵复合。假设用 $R = Q_z(a)Q_x(b)Q_y(c)$ 来表示，其中 $Q_i$ 表示绕物体 $i$ 轴旋转时代表的旋转矩阵。

在第一步中，两种参考系的旋转都是一样的，即 $Q_z=R_z$。

第二步中，绕物体坐标系x轴旋转，这个可以通过先撤销之前z轴的旋转，再绕世界坐标系的x轴旋转，即 $Q_x=Q_z'R_xQ_z$。

最后，同理，$Q_y=Q_x'Q_z'R_yQ_zQ_x$。

合并化简，$Q_zQ_xQ_y=Q_zQ_xQ_x'Q_z'R_yQ_zQ_x=R_yQ_zQ_x=R_yQ_zQ_z'R_xQ_z=R_yR_xR_z$。

发现这一描述和按照世界坐标系yxz次序依次旋转c b a度是等价的。在维基百科[维基百科](https://zh.wikipedia.org/wiki/%E6%AC%A7%E6%8B%89%E8%A7%92)里表示为动态欧拉角的两种表示方法。